In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Bidirectional
from keras.utils import to_categorical
from keras.layers import Flatten

# 데이터 디렉토리 설정
data_dir = "C:/Users/wdd45/OneDrive/바탕 화면/딥러닝응용/human/UCI HAR Dataset/UCI HAR Dataset"

In [14]:
# 데이터 로딩 및 전처리
def load_HAR_data(data_dir):
    train_signals, train_labels, test_signals, test_labels = [], [], [], []
    
    for signal_type in ["train", "test"]:
        for signal in ["Inertial Signals/total_acc_x_", "Inertial Signals/body_acc_x_", "Inertial Signals/body_gyro_x_"]:
            filename = f"{data_dir}/{signal_type}/{signal}{signal_type}.txt"
            data = pd.read_csv(filename, delim_whitespace=True, header=None).values
            if "train" in signal_type:
                train_signals.append(data)
            else:
                test_signals.append(data)

        labels_filename = f"{data_dir}/{signal_type}/y_{signal_type}.txt"
        labels = pd.read_csv(labels_filename, header=None, names=["Activity"]).values
        if "train" in signal_type:
            train_labels.append(labels)
        else:
            test_labels.append(labels)

    train_signals = np.transpose(np.array(train_signals), (1, 2, 0))
    train_labels = train_labels[0]
    test_signals = np.transpose(np.array(test_signals), (1, 2, 0))
    test_labels = test_labels[0]

    train_labels = train_labels - 1
    test_labels = test_labels - 1

    train_signals = (train_signals - np.mean(train_signals)) / np.std(train_signals)
    test_signals = (test_signals - np.mean(test_signals)) / np.std(test_signals)

    return train_signals, train_labels, test_signals, test_labels

In [15]:
train_signals, train_labels, test_signals, test_labels = load_HAR_data(data_dir)

In [16]:
# 데이터를 훈련 및 검증 세트로 분할
X_train, X_val, y_train, y_val = train_test_split(train_signals, train_labels, test_size=0.2, random_state=42)

In [17]:
# One-hot 인코딩
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
test_labels = to_categorical(test_labels)

In [18]:
# BiLSTM 모델
bi_lstm_model = Sequential()
bi_lstm_model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
bi_lstm_model.add(Flatten())
bi_lstm_model.add(Dense(128, activation='relu'))
bi_lstm_model.add(Dense(6, activation='softmax'))
bi_lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

bi_lstm_model.summary()

bi_lstm_history = bi_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

bi_lstm_test_loss, bi_lstm_test_accuracy = bi_lstm_model.evaluate(test_signals, test_labels)
print(f"BiLSTM Test accuracy: {bi_lstm_test_accuracy * 100:.2f}%")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 128, 128)         34816     
 nal)                                                            
                                                                 
 flatten (Flatten)           (None, 16384)             0         
                                                                 
 dense (Dense)               (None, 128)               2097280   
                                                                 
 dense_1 (Dense)             (None, 6)                 774       
                                                                 
Total params: 2,132,870
Trainable params: 2,132,870
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, 

In [30]:
# GRU 모델
gru_model = Sequential()
gru_model.add(GRU(64, input_shape=(X_train.shape[1], X_train.shape[2])))
gru_model.add(Dense(128, activation='relu'))
gru_model.add(Dense(6, activation='softmax'))
gru_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 빌드
gru_model.build((None, X_train.shape[1], X_train.shape[2]))

gru_model.summary()

gru_history = gru_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

gru_test_loss, gru_test_accuracy = gru_model.evaluate(test_signals, test_labels)
print(f"GRU Test accuracy: {gru_test_accuracy * 100:.2f}%")


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_3 (GRU)                 (None, 64)                13248     
                                                                 
 dense_10 (Dense)            (None, 128)               8320      
                                                                 
 dense_11 (Dense)            (None, 6)                 774       
                                                                 
Total params: 22,342
Trainable params: 22,342
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_

In [31]:
# BiGRU 모델
bi_gru_model = Sequential()
bi_gru_model.add(Bidirectional(GRU(64, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
bi_gru_model.add(Flatten())
bi_gru_model.add(Dense(128, activation='relu'))
bi_gru_model.add(Dense(6, activation='softmax'))
bi_gru_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

bi_gru_model.summary()

bi_gru_history = bi_gru_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

bi_gru_test_loss, bi_gru_test_accuracy = bi_gru_model.evaluate(test_signals, test_labels)
print(f"BiGRU Test accuracy: {bi_gru_test_accuracy * 100:.2f}%")

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_4 (Bidirectio  (None, 128, 128)         26496     
 nal)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 16384)             0         
                                                                 
 dense_12 (Dense)            (None, 128)               2097280   
                                                                 
 dense_13 (Dense)            (None, 6)                 774       
                                                                 
Total params: 2,124,550
Trainable params: 2,124,550
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, 